In [1]:
import sys
sys.path.append("C:/Users/parspooya/anaconda3/envs/myNewEnv/Lib/site-packages")

import numpy as np
import pandas as pd
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO
import gym
from gym import spaces
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import configure

In [2]:
# بارگذاری دیتاست
data = pd.read_excel('Book5.xlsx')

In [3]:
# داده‌های ورودی و خروجی
X = data[['p1','k1','s1','t1','a1','p2','k2','s2','t2','a2','p3','k3','s3','t3','a3','p4','k4','s4','t4','a4','p5','k5','s5','t5','a5'
]].values
y = data['task_variant'].values

In [4]:
# تقسیم داده‌ها به داده‌های آموزشی و تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# تعریف محیط 
class CustomEnv(gym.Env):
    def __init__(self, X, y):
        super(CustomEnv, self).__init__()
        self.X = X
        self.y = y
        self.current_step = 0
        self.action_space = spaces.Discrete(3)  # 3 actions: 1, 2, or 3
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(25,), dtype=np.float32)
    
    def reset(self):
        self.current_step = 0
        return self.X[self.current_step]
    
    def step(self, action):
        reward = 1 if action + 1 == self.y[self.current_step] else 0
        self.current_step += 1
        done = self.current_step >= len(self.X)
        obs = self.X[self.current_step] if not done else np.zeros(self.observation_space.shape)
        return obs, reward, done, {}
    
    def render(self, mode='human'):
        pass


In [12]:
del model

NameError: name 'model' is not defined

In [13]:
# ساخت وکتور محیط و آموزش مدل PPO
env = DummyVecEnv([lambda: CustomEnv(X_train, y_train)])

model = PPO(
    'MlpPolicy', 
    env, 
    learning_rate=0.001,   
    gamma=0.99,
    n_steps=2048,
    batch_size=64,
    normalize_advantage=True,
    tensorboard_log='./dqn_tensorboard/' ,
    verbose=1
)


model.learn(total_timesteps=100000, log_interval=4)
# model.save("ppo_25enters")

Using cpu device
Logging to ./dqn_tensorboard/PPO_1
-----------------------------------------
| time/                   |             |
|    fps                  | 658         |
|    iterations           | 4           |
|    time_elapsed         | 12          |
|    total_timesteps      | 8192        |
| train/                  |             |
|    approx_kl            | 0.015231451 |
|    clip_fraction        | 0.244       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.02       |
|    explained_variance   | -0.000337   |
|    learning_rate        | 0.001       |
|    loss                 | 0.946       |
|    n_updates            | 30          |
|    policy_gradient_loss | -0.0227     |
|    value_loss           | 4.24        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 605         |
|    iterations           | 8           |
|    time_elapsed       

In [14]:
# ارزیابی مدل
env = CustomEnv(X_test, y_test)
obs = env.reset()
correct_predictions = 0
total_predictions = 0

# لیست پیش‌بینی‌ها برچسب‌های واقعی
all_predictions = []
all_labels = []

while True:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    correct_predictions += reward
    total_predictions += 1
    all_predictions.append(action + 1) 
    all_labels.append(env.y[env.current_step - 1])
    if done:
        obs = env.reset()
        break



# ارزیابی
accuracy = accuracy_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions, average='weighted')
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')

print(f'Accuracy: {accuracy*100:.2f}%')
print(f'F1 Score: {f1:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')


Accuracy: 92.95%
F1 Score: 0.93
Precision: 0.93
Recall: 0.93
